In [13]:
import numpy as np
from sympy import *
import matplotlib.pyplot as plt
%matplotlib inline
from celmech.nbody_simulation_utilities import set_time_step,align_simulation
from celmech.nbody_simulation_utilities import get_simarchive_integration_results
import rebound as rb
from os.path import isfile
from sympy import init_printing
init_printing()

In [2]:
from celmech.secular import LaplaceLagrangeSystem
from celmech.poincare import Poincare

In [3]:
def eccMatrix(alpha, mass):
    mu1, mu2, mu3 = mass
    P1, P2, P3 = alpha**(3/2), 1, 1/alpha**(3/2)

    sim = rb.Simulation()
    sim.add(m=1.)
    sim.add(m=mu1, P=P1)
    sim.add(m=mu2, P=P2)
    sim.add(m=mu3, P=P3)
    ps = sim.particles
    ps[1].r = ps[1].a*(ps[1].m/3/ps[0].m)**(1/3)
    ps[2].r = ps[2].a*(ps[2].m/3/ps[0].m)**(1/3)

    sim.move_to_com()
    sim.integrator = "whfast"
    sim.dt = sim.particles[1].P/20
    sim.collision = "direct"
    align_simulation(sim)
    
    pvars = Poincare.from_Simulation(sim)
    alpha12 = pvars.particles[1].a/pvars.particles[2].a
    alpha23 = pvars.particles[2].a/pvars.particles[3].a
    alpha13 = pvars.particles[1].a/pvars.particles[3].a
    print(f"alpha = {alpha12, alpha23, alpha13}")

    lsys = LaplaceLagrangeSystem.from_Simulation(sim)  
    M = lsys.Neccentricity_matrix
    print(f"eccentricity matrix =")
    print(M)
    
    return M

In [17]:
def eccMatrix_rotated(alpha, mass):
    m1, m2, m3 = mass
    m_tot = m1+m2+m3
    
    R = np.array([[np.sqrt(m1/m_tot), np.sqrt(m2/m_tot), np.sqrt(m3/m_tot)],
             [-np.sqrt(m3/(m1+m3)), 0, np.sqrt(m1/(m1+m3))],
             [np.sqrt(m1*m2/(m1+m3)/m_tot), -np.sqrt((m1+m3)/m_tot), np.sqrt(m2*m3/(m1+m3)/m_tot)]])
    M = eccMatrix(alpha, mass)
    
    vals, k = np.linalg.eigh(M)
    print("diagonalized")
    print(vals)
    
    return np.matmul(np.matmul(R, M), np.transpose(R))

In [19]:
eccMatrix_rotated(0.9, (1e-8, 1e-10, 1e-8))

alpha = (0.8999999754755369, 0.899999981291701, 0.8099999610905145)
eccentricity matrix =
[[-2.35358584e-07  9.27220801e-08  1.93764278e-07]
 [ 9.27220801e-08 -1.77628054e-06  7.91675071e-08]
 [ 1.93764278e-07  7.91675071e-08 -2.10935812e-07]]
diagonalized
[-1.78477587e-06 -4.17167272e-07 -2.06318020e-08]


array([[-2.09705115e-08,  1.15049309e-08,  2.57486995e-09],
       [ 1.15049309e-08, -4.16911477e-07,  1.04219834e-08],
       [ 2.57486995e-09,  1.04219834e-08, -1.78469295e-06]])